In [45]:
import pandas as pd
import re

from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
from random import randint
from time import sleep, ctime

import psycopg2
from sqlalchemy import create_engine
from config import driver, username, password, host, port, database

def init_browser():
    executable_path = {'executable_path': ChromeDriverManager().install()}
    return Browser('chrome', **executable_path, headless=False)

In [46]:
# Cities to search
cities = ['Los-Angeles_CA','New-York_NY', 'Chicago_IL', 'Houston_TX']

cities = ['Duarte_CA']

In [47]:
# Start browser
browser = init_browser()



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\kesam\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


In [48]:
# Create blank lists/dictionaries to store attributes
prices = []
beds = []
baths = []
sizes = []
addresses = []
statuses = []
detail_pages = []

In [49]:
listing_indicators = {}
types = []
fees = []
pricesqfts = []
garages = []
years = []

In [50]:
## Identify listing attributes from result card

In [51]:
# Loop through each city
for city in cities:

    print(f"Searching {city}...{ctime()}\n")
    
    # Loop through each search result page
    for i in range(2, 3, 1):
        
        # Set dynamic URL
        url = f"https://www.realtor.com/realestateandhomes-search/{city}/pg-{i}"
        browser.visit(url)
        
        print(f"Scraping page {i} at {ctime()}")
        
        # HTML object
        html = browser.html
        
        # Parse HTML with Beautiful Soup
        soup = bs(html, "html.parser")

        # Identify all listings
        listings = soup.find_all('li', attrs={"data-testid": "result-card"})

        # Loop through each listing to identify attributes
        for listing in listings:          
            try:
                price = listing.find('span', attrs={"data-label": "pc-price"}).text.strip('$')
                prices.append(price.replace(",",""))
            except:
                prices.append('No Info')
            try:
                bed = int(listing.find('li', attrs={"data-label": "pc-meta-beds"}).text.strip('bed'))
                beds.append(bed)
            except:
                beds.append('No Info')
            try:
                bath = float(listing.find('li', attrs={"data-label": "pc-meta-baths"}).text.strip()[0])
                baths.append(bath)
            except:
                baths.append('No Info')
            try:
                size = listing.find('li', attrs={"data-label": "pc-meta-sqft"}).text.strip('sqft')
                sizes.append(size)
            except:
                sizes.append('No Info')
            try:
                address = listing.find('div', attrs={"data-label": "pc-address"}).text
                addresses.append(address)
            except:
                addresses.append('No Info')
            try:
                status = listing.find('span', attrs={"class": "jsx-3853574337 statusText"}).text
                statuses.append(status)
            except:
                statuses.append('No Info')

            
            # Identify URL to listing detail page
            detail_page = listing.find('a').get('href')
            
            # Append to list
            detail_pages.append(detail_page)
            
        # Generate random number between 2 to 10 seconds to wait before continuing loop
        sleep(randint(2,10))
            
    print(f"\n----------------------------{ctime()}\n")
    
print('Scraping complete')

Searching Duarte_CA...Sat Sep 25 10:07:09 2021

Scraping page 2 at Sat Sep 25 10:07:12 2021

----------------------------Sat Sep 25 10:07:27 2021

Scraping complete


In [8]:
## Identify listing attributes from detail page

In [52]:
i = 1
num_page = len(detail_pages)
print(f"Total of {num_page} listings found\n")

# Loop through each listing detail page
for detail_page in detail_pages:
    
    print(f"Scraping details from listing {i} of {num_page}")
    
    # Navigate to each href
    detail_url = f"https://www.realtor.com{detail_page}"
    
    browser.visit(detail_url)
    
    # HTML object
    html = browser.html

    # Parse HTML with Beautiful Soup
    detail_soup = bs(html, "html.parser")

    # Identify all listing-indicators
    details = detail_soup.find('div', attrs={"data-testid": "listing-indicator"})
    re_li = re.compile('rui*')
    
    try:
        for x in details.find_all('li', re_li):
            listing_indicators[x.find_all('div', attrs={'class': re_li})[0].text] = x.find_all('div', attrs={'class': re_li})[1].text
            
        try:
            property_type = listing_indicators['Property Type']
            types.append(property_type)
        except:
            types.append('No Info')
        try:
            hoa_fee = listing_indicators['HOA Fees'].strip('/mo')
            hoa_fee = hoa_fee.strip('$')
            fees.append(hoa_fee)
        except:
            fees.append('No Info')
        try:
            pricesqft = listing_indicators['Price per sqft'].strip('$')
            pricesqfts.append(pricesqft)
        except:
            pricesqfts.append('No Info') 
        try:
            garage = listing_indicators['Garage'].strip(' car')
            garage = garage.strip(' cars')
            garages.append(garage)
        except:
            garages.append('No Info')
        try:
            year = listing_indicators['Year Built']
            years.append(year)
        except:
            years.append('No Info')

    except:
        types.append('No Info')
        fees.append('No Info')
        pricesqfts.append('No Info')
        garages.append('No Info')
        years.append('No Info')
        
        sleep(randint(2,15))
    
    i = i + 1
    
    # Generate random number between 2 to 10 seconds to wait before continuing loop
    # sleep(randint(2,10))
    
print('\nScraping complete')

Total of 4 listings found

Scraping details from listing 1 of 4
Scraping details from listing 2 of 4
Scraping details from listing 3 of 4
Scraping details from listing 4 of 4

Scraping complete


In [53]:
# Add attributes to dataframe
df = pd.DataFrame({'Address': addresses, 'Status': statuses, 'Property Type': types, 'Price': prices, 'Price per sqft': pricesqfts, 'HOA Fees': fees, 'Bed': beds, 'Bath': baths, 'Size': sizes, 'Garage': garages, 'Year Built': years})

# Extract address into Street, City, State, Zip
street_city = df['Address'].str.split(',', expand=True)
street_city = street_city.rename(columns={0: 'Street', 1: 'City', 2: 'state_zip'})
state_zip = street_city['state_zip'].str.split(' ', expand=True)
state_zip = state_zip.rename(columns={1: 'State', 2: 'Zip'})
street_city = street_city.drop(columns='state_zip')
state_zip = state_zip.drop(columns=0)

street_city.reset_index(drop=True, inplace=True)
state_zip.reset_index(drop=True, inplace=True)

# Create merged_df
address_df = pd.concat([street_city, state_zip], axis=1) 

merged_df = pd.concat([df, address_df], axis=1) 

In [44]:
merged_df.to_csv('listings.csv', index=False)

In [ ]:
browser.quit()

In [54]:
merged_df.head()

,Address,Status,Property Type,Price,Price per sqft,HOA Fees,Bed,Bath,Size,Garage,Year Built,Street,City,State,Zip
0,"Opal Cyn, Duarte, CA 91010",Contingent,No Info,399000,No Info,No Info,No Info,No Info,No Info,No Info,No Info,Opal Cyn,Duarte,CA,91010
1,"1820 Huntington Dr, Duarte, CA 91010",Pending,No Info,475000,No Info,No Info,2,2.0,"1,060",No Info,No Info,1820 Huntington Dr,Duarte,CA,91010
2,"2074 Goodall Ave, Duarte, CA 91010",Pending,No Info,545000,No Info,No Info,3,1.0,927,No Info,No Info,2074 Goodall Ave,Duarte,CA,91010
3,"3514 Conata St, Duarte, CA 91010",Pending,Single Family,699950,448,No Info,4,2.0,"1,564",2,1966,3514 Conata St,Duarte,CA,91010


In [55]:
# Setup database connection
connection_string = f"{driver}://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)
connection = engine.connect()

In [56]:
listings_df = merged_df.rename(columns= {'Address':'full_address',
                                          'Status':'status', 
                                          'Property Type': 'property_type',
                                          'Price':'price',
                                          'Price per sqft':'price_per_sq_ft',
                                          'HOA Fees':'hoa_fees',
                                          'Bed':'bed',
                                          'Bath':'bath',
                                          'Size':'built_area',
                                          'Garage':'parking_garage',
                                          'Year Built':'year_built',
                                          'Street':'street_address',
                                          'City':'city',
                                          'State':'state_cd',
                                          'Zip':'zip_code'
                                 })
listings_df['load_datetime'] = ctime()
listings_df

,full_address,status,property_type,price,price_per_sq_ft,hoa_fees,bed,bath,built_area,parking_garage,year_built,street_address,city,state_cd,zip_code,load_datetime
0,"Opal Cyn, Duarte, CA 91010",Contingent,No Info,399000,No Info,No Info,No Info,No Info,No Info,No Info,No Info,Opal Cyn,Duarte,CA,91010,Sat Sep 25 10:08:15 2021
1,"1820 Huntington Dr, Duarte, CA 91010",Pending,No Info,475000,No Info,No Info,2,2.0,"1,060",No Info,No Info,1820 Huntington Dr,Duarte,CA,91010,Sat Sep 25 10:08:15 2021
2,"2074 Goodall Ave, Duarte, CA 91010",Pending,No Info,545000,No Info,No Info,3,1.0,927,No Info,No Info,2074 Goodall Ave,Duarte,CA,91010,Sat Sep 25 10:08:15 2021
3,"3514 Conata St, Duarte, CA 91010",Pending,Single Family,699950,448,No Info,4,2.0,"1,564",2,1966,3514 Conata St,Duarte,CA,91010,Sat Sep 25 10:08:15 2021


In [57]:
listings_df.to_sql('listings',connection, if_exists='append', index=False)

In [31]:
listings_df.head()

,full_address,status,property_type,price,price_per_sq_ft,hoa_fees,bed,bath,built_area,parking_garage,year_built,street_address,city,state_cd,zip_code,load_datetime
0,"Opal Cyn, Duarte, CA 91010",Contingent,Land,"399,000",No Info,No Info,No Info,No Info,No Info,No Info,No Info,Opal Cyn,Duarte,CA,91010,Fri Sep 24 23:54:10 2021
1,"1820 Huntington Dr, Duarte, CA 91010",Pending,Condo,"475,000",448,412,2,2.0,"1,060",2,1985,1820 Huntington Dr,Duarte,CA,91010,Fri Sep 24 23:54:10 2021
2,"2074 Goodall Ave, Duarte, CA 91010",Pending,Single Family,"545,000",588,412,3,1.0,927,1,1952,2074 Goodall Ave,Duarte,CA,91010,Fri Sep 24 23:54:10 2021
3,"3514 Conata St, Duarte, CA 91010",Pending,Single Family,"699,950",448,412,4,2.0,"1,564",2,1966,3514 Conata St,Duarte,CA,91010,Fri Sep 24 23:54:10 2021


In [58]:
new_listings_df = pd.read_sql_table('listings', connection)
new_listings_df

,full_address,status,property_type,price,price_per_sq_ft,hoa_fees,bed,bath,built_area,parking_garage,year_built,street_address,city,state_cd,zip_code,property_id,load_datetime
0,"Opal Cyn, Duarte, CA 91010",Contingent,No Info,399000.0,No Info,No Info,No Info,No Info,No Info,No Info,No Info,Opal Cyn,Duarte,CA,91010,1228,2021-09-25 10:08:15
1,"1820 Huntington Dr, Duarte, CA 91010",Pending,No Info,475000.0,No Info,No Info,2,2.0,"1,060",No Info,No Info,1820 Huntington Dr,Duarte,CA,91010,1229,2021-09-25 10:08:15
2,"2074 Goodall Ave, Duarte, CA 91010",Pending,No Info,545000.0,No Info,No Info,3,1.0,927,No Info,No Info,2074 Goodall Ave,Duarte,CA,91010,1230,2021-09-25 10:08:15
3,"3514 Conata St, Duarte, CA 91010",Pending,Single Family,699950.0,448,No Info,4,2.0,"1,564",2,1966,3514 Conata St,Duarte,CA,91010,1231,2021-09-25 10:08:15
4,"4343 S Budlong Ave, Los Angeles,CA,90037",For Sale,Single Family,749000.0,446,No Info,4,3,"1,680",No Info,1922,4343 S Budlong Ave,Los Angeles,CA,90037,389,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838,"415 Byrne St, Houston,TX,77009",For Sale,Single Family,575000.0,251,66,3,3,"2,287",2,1920,415 Byrne St,Houston,TX,77009,1223,NaT
839,"2207 Bancroft St Apt 1204, Houston,TX,77027",For Sale,Condo,749000.0,548,1147,2,2,"1,366",2,2010,2207 Bancroft St Apt 1204,Houston,TX,77027,1224,NaT
840,"12523 Malachite Way, Houston,TX,77077",For Sale,Single Family,399995.0,181,183,4,3,"2,213",2,2020,12523 Malachite Way,Houston,TX,77077,1225,NaT
841,"14918 LA Quinta Ln, Houston,TX,77079",For Sale,Single Family,629900.0,294,83,4,2,"2,143",2,1973,14918 LA Quinta Ln,Houston,TX,77079,1226,NaT
